In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import json
import re
import pickle

from vietunits.utils import key_normalize

import warnings
warnings.filterwarnings('ignore')

BASE_DIR = Path().resolve().parent.parent

In [2]:
# IDEA
data = {
    'thanhphohochiminh': {'keywords': ['thanhphohochiminh', 'hochiminh', 'hcm']},
    'hanoi': {'keywords': ['hanoi', 'hn']},
}

# Bước 1: Tạo list keyword, sắp xếp theo độ dài giảm dần
keywords = sorted(
    [kw for v in data.values() for kw in v['keywords']],
    key=len,
    reverse=True
)

# Bước 2: Tạo regex pattern
pattern = re.compile('|'.join(re.escape(k) for k in keywords), flags=re.IGNORECASE)

# Bước 3: Kiểm tra chuỗi address
address = 'quan5, hcm'

match = pattern.search(address)
keyword = match.group(0) if match else None

# Bước 4: Tìm key trong data tương ứng với keyword
data_key = next((k for k, v in data.items() if keyword and keyword.lower() in [kw.lower() for kw in v['keywords']]), None)

print(data_key)

thanhphohochiminh


In [3]:
def create_sort(text, level=1):
    if isinstance(text, str):
        if level == 1:
            text = re.sub(r'^Tỉnh\s|Thành phố\s', '', text, flags=re.IGNORECASE)
        elif level == 2:
            if re.search(r'^Quận\s\d{1,2}', text, flags=re.IGNORECASE):
                pass
            else:
                text = re.sub(r'^Quận\s|Huyện\s|Thị xã\s|Thành phố\s', '', text, flags=re.IGNORECASE)
        else:
            if re.search(r'^Phường\s\d{1,2}', text, flags=re.IGNORECASE):
                pass
            else:
                text = re.sub(r'^Phường\s|Thị trấn\s|Xã\s', '', text, flags=re.IGNORECASE)

        return text.strip()
    return text

district_type_acronym = {
    'Quận': 'q',
    'Thị xã': 'tx',
    'Thành phố': 'tp',
    'Huyện': 'h',
}
ward_type_acronym = {
    'Phường': 'p',
    'Thị trấn': 'tt',
    'Xã': 'x'
}
def create_keywords(row, level=1):
    keywords = []
    if level == 1:
        keywords.append(row['provinceKey'])
        keywords.append(row['provinceShortKey'])
        if pd.notnull(row['provinceAlias']):
            aliases = json.loads(row['provinceAlias'])
            for a in aliases:
                keywords.append(key_normalize(a))

    elif level == 2:
        keywords.append(row['districtKey'])
        if not row['districtShortKeyDuplicated']:
            keywords.append(row['districtShortKey'])
        else:
            keywords.append(key_normalize(f"{row['districtShortKey']} {row['districtType']}"))
            keywords.append(key_normalize(f"{district_type_acronym[row['districtType']]}.{row['districtShortKey']}"))

        if pd.notnull(row['districtAlias']):
            aliases = json.loads(row['districtAlias'])
            for a in aliases:
                keywords.append(key_normalize(a))

        if re.search(r'^quan\d{1,2}', row['districtKey'], flags=re.IGNORECASE):
            keywords.append(row['districtKey'].replace('quan', 'q'))
            keywords.append(row['districtKey'].replace('quan', 'q.'))
            keywords.append(row['districtKey'].replace('quan', 'district'))

    else:
        if pd.notnull(row['wardKey']):
            keywords.append(row['wardKey'])
            if not row['wardShortKeyDuplicated']:
                keywords.append(row['wardShortKey'])
            else:
                keywords.append(key_normalize(f"{row['wardShortKey']} {row['wardType']}"))
                keywords.append(key_normalize(f"{ward_type_acronym[row['wardType']]}.{row['wardShortKey']}"))

            if pd.notnull(row['wardAlias']):
                aliases = json.loads(row['wardAlias'])
                for a in aliases:
                    keywords.append(key_normalize(a))

            if re.search(r'^phuong\d{1,2}', row['wardKey'], flags=re.IGNORECASE):
                keywords.append(row['wardKey'].replace('phuong', 'p'))
                keywords.append(row['wardKey'].replace('phuong', 'p.'))
                keywords.append(row['wardKey'].replace('phuong', 'ward'))
        else:
            return np.nan

    keywords = list(set(keywords))
    keywords = sorted(keywords, key=len, reverse=True)
    return json.dumps(keywords)

In [4]:
df_convert = pd.read_csv(BASE_DIR / 'data/danhmuc_and_sapnhap_has_default_new_ward.csv')
cols = [
    'provinceCode',
    'districtCode',
    'districtType',
    'wardCode',
    'wardType',
    'province',
    'district',
    'ward'
]
df = df_convert[cols].drop_duplicates().reset_index(drop=True)

In [5]:
# ENRICH DATA
unit_cols = ['province', 'district', 'ward']
level_map = {
    'province': 1,
    'district': 2,
    'ward': 3
}

for col in unit_cols:
    # Create short version
    level = level_map[col]
    df[f"{col}Short"] = df[col].apply(create_sort, args=(level,))

    # Create key
    df[f"{col}Key"] = df[f"{col}"].apply(key_normalize)

    # Create short key
    df[f"{col}ShortKey"] = df[f"{col}Short"].apply(key_normalize)

In [6]:
# -- CREATE ALIAS
# Khởi tạo cột alias rỗng
for col in ['province', 'district', 'ward']:
    df[f"{col}Alias"] = np.nan

# Province alias data
province_alias_data = {
    'thanhphohanoi': ['hn'],
    'thanhphohochiminh': ['hcm'],
    'tinhbariavungtau': ['baria', 'vungtau'],
}

# District alias data (theo từng province)
district_alias_data = {
    'thanhphohochiminh': {
        'thanhphothuduc': ['quan9', 'quan2']
    }
}

# Ward alias data (theo từng province > district > ward)
ward_alias_data = {
    # 'thanhphohochiminh': {
    #     'thanhphothuduc': {
    #         'phuongtruongthanh': ['truongthanh', 'p.truongthanh']
    #     }
    # }
}

# Gán provinceAlias
for key, value in province_alias_data.items():
    df.loc[df['provinceKey'] == key, 'provinceAlias'] = json.dumps(value)

# Gán districtAlias
for province_key, district_data in district_alias_data.items():
    for district_key, value in district_data.items():
        df.loc[
            (df['provinceKey'] == province_key) & (df['districtKey'] == district_key),
            'districtAlias'
        ] = json.dumps(value)

# Gán wardAlias
for province_key, district_data in ward_alias_data.items():
    for district_key, ward_data in district_data.items():
        for ward_key, value in ward_data.items():
            df.loc[
                (df['provinceKey'] == province_key) &
                (df['districtKey'] == district_key) &
                (df['wardKey'] == ward_key),
                'wardAlias'
            ] = json.dumps(value)

In [7]:
# Find duplicate district keys
df_district = df[['province', 'provinceKey', 'district', 'districtKey', 'districtShortKey']].drop_duplicates()

In [8]:
# Check district key
df_district.groupby(['province', 'districtKey']).size().reset_index(name='count').sort_values(by=['count'], ascending=False).head()
# District key is unique

,province,districtKey,count
0,Thành phố Cần Thơ,huyencodo,1
467,Tỉnh Quảng Nam,huyenphuninh,1
459,Tỉnh Quảng Nam,huyenbactramy,1
460,Tỉnh Quảng Nam,huyendailoc,1
461,Tỉnh Quảng Nam,huyendonggiang,1


In [9]:
# Check district short key
count_district_short_key = df_district.groupby(['province', 'districtShortKey']).size().reset_index(name='count').sort_values(by=['count'], ascending=False)
duplicated_district_short_key = count_district_short_key[count_district_short_key['count']>1].copy()
duplicated_district_short_key['districtShortKeyDuplicated'] = True
duplicated_district_short_key.drop(columns=['count'], inplace=True)

df = pd.merge(df, duplicated_district_short_key, on=['province', 'districtShortKey'], how='left')
df['districtShortKeyDuplicated'].fillna(False, inplace=True)

In [10]:
# Find duplicated ward short key
df_ward = df[['province', 'district', 'wardKey', 'wardShortKey']].drop_duplicates()

In [11]:
# Check wardKey
df_ward.groupby(['province', 'district', 'wardKey']).size().reset_index(name='count').sort_values(by=['count'], ascending=False).head()
# wardKey is unique

,province,district,wardKey,count
0,Thành phố Cần Thơ,Huyện Cờ Đỏ,thitrancodo,1
6679,Tỉnh Quảng Nam,Thành phố Hội An,phuongcaman,1
6672,Tỉnh Quảng Nam,Huyện Đại Lộc,xadainghia,1
6673,Tỉnh Quảng Nam,Huyện Đại Lộc,xadaiphong,1
6674,Tỉnh Quảng Nam,Huyện Đại Lộc,xadaiquang,1


In [12]:
# Check duplicated short key
count_ward_short_key = df_ward.groupby(['province', 'district', 'wardShortKey']).size().reset_index(name='count').sort_values(by=['count'], ascending=False)
duplicated_ward_short_key = count_ward_short_key[count_ward_short_key['count']>1].copy()
duplicated_ward_short_key['wardShortKeyDuplicated'] = True
duplicated_ward_short_key.drop(columns=['count'], inplace=True)

df = pd.merge(df, duplicated_ward_short_key, on=['province', 'district', 'wardShortKey'], how='left')
df['wardShortKeyDuplicated'].fillna(False, inplace=True)

In [13]:
# Create keywords
for col in unit_cols:
    level = level_map[col]
    df[f"{col}Keywords"] = df.apply(lambda row: create_keywords(row, level=level), axis=1)

In [14]:
# Province map
df_province = df[['provinceKey', 'provinceKeywords', 'province', 'provinceShort']].drop_duplicates().reset_index(drop=True)
DICT_PROVINCE = {}
for _, row in df_province.iterrows():
    DICT_PROVINCE[row['provinceKey']] = {
        'provinceKeywords': json.loads(row['provinceKeywords']),
        'province': row['province'],
        'provinceShort': row['provinceShort'],
    }


# District map
df_district = df[['provinceKey', 'provinceShortKey', 'districtKey', 'districtShortKey', 'districtKeywords', 'district', 'districtType', 'districtShort']].drop_duplicates().reset_index(drop=True)
DICT_PROVINCE_DISTRICT = {}
for _, province_row in df_province.iterrows():
    province_key = province_row['provinceKey']
    DICT_PROVINCE_DISTRICT[province_key] = {}

    df_district_filtered = df_district[df_district['provinceKey'] == province_key]

    for _, district_row in df_district_filtered.iterrows():
        DICT_PROVINCE_DISTRICT[province_key][district_row['districtKey']] = {
            'districtKeywords': json.loads(district_row['districtKeywords']) if pd.notnull(district_row['districtKeywords']) else [],
            'district': district_row['district'],
            'districtType': district_row['districtType'],
            'districtShort': district_row['districtShort'],
        }


# Unique district to province map
province_short_keys = df['provinceShortKey'].unique().tolist()
for index, row in df_district.iterrows():
    district_short_key = row['districtShortKey']
    left_district_short_keys = df_district.loc[df_district.index != index, 'districtShortKey'].tolist()
    if district_short_key not in province_short_keys and district_short_key not in left_district_short_keys:
        df_district.loc[index, 'districtUnique'] = True
df_district['districtUnique'].fillna(False, inplace=True)
DICT_UNIQUE_DISTRICT_PROVINCE = {}
for _, row in df_district.iterrows():
    DICT_UNIQUE_DISTRICT_PROVINCE[row['districtKey']] = {
        'districtKeywords': json.loads(row['districtKeywords']),
        'provinceKey': row['provinceKey']
    }


# Ward map
df_ward = df[['provinceKey', 'districtKey', 'wardKey', 'wardKeywords', 'ward', 'wardShort', 'wardType']].drop_duplicates().reset_index(drop=True)
DICT_PROVINCE_DISTRICT_WARD = {}
for province_key, province_group in df_ward.groupby('provinceKey'):
    DICT_PROVINCE_DISTRICT_WARD[province_key] = {}

    for district_key, district_group in province_group.groupby('districtKey'):
        DICT_PROVINCE_DISTRICT_WARD[province_key][district_key] = {}

        for _, row in district_group.iterrows():
            ward_key = row['wardKey']
            DICT_PROVINCE_DISTRICT_WARD[province_key][district_key][ward_key] = {
                'wardKeywords': json.loads(row['wardKeywords']) if pd.notnull(row['wardKeywords']) else [],
                'ward': row['ward'],
                'wardShort': row['wardShort'],
                'wardType': row['wardType'],
            }

In [15]:
pickle_data = {
    'DICT_PROVINCE': DICT_PROVINCE,
    'DICT_PROVINCE_DISTRICT': DICT_PROVINCE_DISTRICT,
    'DICT_UNIQUE_DISTRICT_PROVINCE': DICT_UNIQUE_DISTRICT_PROVINCE,
    'DICT_PROVINCE_DISTRICT_WARD': DICT_PROVINCE_DISTRICT_WARD
}

In [16]:
with open(BASE_DIR / 'vietunits/data/v63provinces/pickle_data.pkl', 'wb') as f:
    pickle.dump(pickle_data, f)